##### Initialize

In [2]:
import sys
sys.path.append('../')

from msal import ConfidentialClientApplication
import pandas as pd
import os
from azure.keyvault.secrets import SecretClient
from azure.identity import DefaultAzureCredential

import logging

def_credential = DefaultAzureCredential()  

## connect to Azure Key Vault securely
keyVaultName = os.environ["KEY_VAULT_NAME"]
KVUri = f"https://{keyVaultName}.vault.azure.net"
kv_client = SecretClient(vault_url=KVUri, credential=def_credential)

## Secret Keys for Azure App Registration
tenant_id = kv_client.get_secret('azure-tenant-id').value
client_id = kv_client.get_secret('idgov-app-client-id').value
client_secret = kv_client.get_secret('idgov-app-client-secret').value

## Secret Keys for Bolddesk (Nera Care)
# bd_api_key = kv_client.get_secret ('bolddesk-nera-care-api-key').value
# bd_base_url = kv_client.get_secret('bolddesk-nera-care-api-base-url').value
## Secret Keys for Bolddesk (Nera IT)
bd_api_key = kv_client.get_secret ('bolddesk-nera-care-api-key').value
bd_base_url = kv_client.get_secret('bolddesk-nera-care-api-base-url').value

In [3]:
from module.bolddesk import Bolddesk
from module.warehouse import Warehouse
from module.azure_ad import AzureAD
## Initialize AD and Bolddesk API Module
ad = AzureAD(tenant_id, client_id, client_secret)
bd = Bolddesk(bd_base_url, bd_api_key)
wh = Warehouse(
        server=os.environ["DB_SERVER"],
        database=os.environ["DB_NAME"],
        credential=def_credential
)

#### Tickets

In [11]:
df = bd.list_tickets()
table_name = 'bd_nera_care_tickets'
wh.erase(table_name)
wh.append(table_name, df)

#### Contacts

In [9]:
df = bd.list_contacts()
table_name = 'bd_nera_care_contacts'
wh.erase(table_name)
wh.append(table_name, df)

#### Agents

In [10]:
df = bd.list_agents()
table_name = 'bd_nera_care_agents'
wh.erase(table_name)
wh.append(table_name, df)

#### Convert All Active Contacts to Agent with Role "Approval"

In [3]:
# contacts_df = bd.list_contacts()
# active = ~contacts_df.isBlocked & ~contacts_df.isDeleted & contacts_df.emailId.str.contains('@nera.net')
# active_contacts_df = contacts_df.loc[active]

# active_contacts_df
# for idx, contact in active_contacts_df.iterrows():
#     update_data = {
#         "name": contact.get('contactName'),
#         "displayName": contact.get('contactName'),
#         "hasAllBrandAccess": True,
#         "brandIds": "1",          ## default brand
#         "roleIds": "1002",        ## approval
#         "ticketAccessScopeId": 2  ## group 
#     }
#     print(update_data)
#     bd.convert_to_agent(contact.userId, update_data)


#### Deacticate Automatically

In [3]:
ad_users_df = ad.list_users().query('userPurpose=="user" and userType=="Member" and accountEnabled==True').set_index('userPrincipalName')
agents_df = bd.list_agents().set_index('emailId')
invalid_agents = ~agents_df.index.isin(ad_users_df.index)  # & ~ (agents_df.isBlocked | agents_df.isDeleted) & ~agents_df.index.isin(valid_non_nera_domains)
invalid_agents_df = agents_df.loc[invalid_agents]

if len(invalid_agents_df)/len(ad.users_df) < 0.1:
    for emailId, agent in invalid_agents_df.iloc[:3].iterrows():
        bd.deactivate_agent(agent.userId)

In [5]:
bd.deactivate_agent(1045)

{'message': 'Agent successfully deactivated.'}

In [5]:
agent

name                                            Ahmed Zia Arshad
displayName                                     Ahmed Zia Arshad
userId                                                      1035
roles                [{'roleName': 'Approvals', 'roleId': 1002}]
status                                                    Active
lastActivityOn                                              None
shortCode                                                     AZ
colorCode                                                #009688
ticketAccessScope                                          Group
isAvailable                                                 True
isVerified                                                  True
createdOn                                   2023-04-23T09:07:42Z
lastModifiedOn                              2023-06-22T09:41:02Z
Name: ahmed.zia@nera.net, dtype: object

#### Manually Verify All Non-Verified Agents

In [10]:
# agents_df = bd.list_agents() 
# agents_df = agents_df.loc[agents_df.isVerified==False]

# for idx, user in agents_df.iterrows():
#     print('verify user: ', user.userId)
#     bd.verify_user(user.userId)

verify user:  1021
verify user:  1184
verify user:  1494
verify user:  1520
verify user:  1025
verify user:  1027
verify user:  1031
verify user:  1032
verify user:  1033
verify user:  1495
verify user:  1183
verify user:  1209
verify user:  1472
verify user:  1034
verify user:  1035
verify user:  1515
verify user:  1039
verify user:  1042
verify user:  1043
verify user:  1045
verify user:  1049
verify user:  1050
verify user:  1046
verify user:  1047
verify user:  1048
verify user:  1051
verify user:  1053
verify user:  1054
verify user:  1246
verify user:  1377
verify user:  1055
verify user:  1056
verify user:  1057
verify user:  1058
verify user:  1059
verify user:  1060
verify user:  1061
verify user:  1496
verify user:  1062
verify user:  1063
verify user:  1064
verify user:  1065
verify user:  1068
verify user:  1069
verify user:  1066
verify user:  1070
verify user:  1071
verify user:  1497
verify user:  1072
verify user:  1073
verify user:  1074
verify user:  1075
verify user:

#### Update All Active Agents With Role 1002 (Approval)

In [6]:
# agents_df = bd.list_agents()
# active = agents_df.status=='Active'
# for idx, agent in agents_df.loc[active].iterrows():
#     roles = [ str(r.get('roleId')) for r in agent.roles]    
#     ## Approval Role not assigned
#     if '1002' not in roles:
#         roles_string = ','.join(roles) + ',1002'
#         agent_update = {
#             'roleIds': roles_string,
#             'ticketAccessScopeId': 1 if (agent.get('ticketAccessScope')=='Global') else 2,
#             'displayName': agent.get('displayName'), ## no change
#             'emailId': agent.get('emailId'),         ## no change
#             'name': agent.get('name'),               ## no change
#         }
#         print('Update Agent To Approval Role: ', agent.get('name'), ' >> ', agent_update)
#         bd.update_agent(agent.userId, agent_update)

Update Agent To Approval Role:  Alvin Mah  >>  {'roleIds': '1,1002', 'ticketAccessScopeId': 1, 'displayName': 'Alvin Mah', 'emailId': 'alvin.mah@nera.net', 'name': 'Alvin Mah'}
Update Agent To Approval Role:  Nera BOT  >>  {'roleIds': '1000,1002', 'ticketAccessScopeId': 1, 'displayName': 'Nera BOT', 'emailId': 'nera-bot@nera.net', 'name': 'Nera BOT'}
Update Agent To Approval Role:  Vera Ong  >>  {'roleIds': '1003,1002', 'ticketAccessScopeId': 2, 'displayName': 'Vera Ong', 'emailId': 'vera.ong@nera.net', 'name': 'Vera Ong'}
